# Part 3: Unbiased Evaluation using a New Test Set

In this part, we are given a new test set (`/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`). We can now take advantage of the entire smart sample that we created in Part I. 

* Retrain a pipeline using the optimal parameters that the pipeline learned. We don't need to repeat GridSearch here. 

## Import modules as needed

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd

from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

## Load smart sample and the best pipeline from Part II

In [32]:
import joblib

X_sampled, y_sampled, pipe3, grid_model3 = joblib.load('model-v1.pkl')


##  Retrain a pipeline using the full sampled training data set

Use the full sampled training data set to train the pipeline.

In [33]:
# Add code below this comment  (Question #E301)
# ----------------------------------

model = grid_model3.fit(X_sampled,y_sampled)




### Save the trained model with the pickle library.

In [34]:
# Add code below this comment  
# -----------------------------

joblib.dump([model],'newmodel-v1.pkl')




['newmodel-v1.pkl']

In [35]:

del X_sampled, y_sampled


## Load the Testing Data and evaluate your model

 * `/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`
 
* We need to preprocess this test data (follow the steps similar to Part I)
* If we have fitted any normalizer/standardizer in Part 2, then we have to transform this test data using the fitted normalizer/standardizer

In [6]:
# Preprocess the given test set  (Question #E302)
# ----------------------------------
DATASET = '/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)

dataset.head().transpose()






/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
sku,3351237,3513721,3432072,3354657,3489659
national_inv,8.0,10.0,3.0,26.0,0.0
lead_time,8.0,12.0,2.0,2.0,8.0
in_transit_qty,0.0,0.0,0.0,0.0,0.0
forecast_3_month,20.0,0.0,0.0,25.0,0.0
forecast_6_month,20.0,0.0,0.0,70.0,0.0
forecast_9_month,20.0,0.0,0.0,110.0,0.0
sales_1_month,0.0,0.0,0.0,7.0,0.0
sales_3_month,0.0,0.0,0.0,35.0,0.0
sales_6_month,0.0,0.0,0.0,64.0,0.0


In [7]:
dataset = dataset.iloc[:,1:]
dataset.isnull().sum()

national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          1
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                1
oe_constraint            1
ppap_risk                1
stop_auto_buy            1
rev_stop                 1
went_on_backorder        1
dtype: int64

In [8]:
dataset['lead_time'].fillna(int(dataset['lead_time'].mode()),inplace=True)

yes_no_columns = list(filter(lambda i: dataset[i].dtype!=np.float64, dataset.columns))
for column_name in yes_no_columns:
    mode = dataset[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    dataset[column_name].fillna(mode, inplace=True)

for i in yes_no_columns:
    
    dataset[i] = list(map(['Yes', 'No'].index, dataset[i]))

Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No


In [9]:
dataset.isnull().sum()
dataset[dataset.isna().any(axis=1)]
dataset.dropna(inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242075 entries, 0 to 242075
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   national_inv       242075 non-null  float64
 1   lead_time          242075 non-null  float64
 2   in_transit_qty     242075 non-null  float64
 3   forecast_3_month   242075 non-null  float64
 4   forecast_6_month   242075 non-null  float64
 5   forecast_9_month   242075 non-null  float64
 6   sales_1_month      242075 non-null  float64
 7   sales_3_month      242075 non-null  float64
 8   sales_6_month      242075 non-null  float64
 9   sales_9_month      242075 non-null  float64
 10  min_bank           242075 non-null  float64
 11  potential_issue    242075 non-null  int64  
 12  pieces_past_due    242075 non-null  float64
 13  perf_6_month_avg   242075 non-null  float64
 14  perf_12_month_avg  242075 non-null  float64
 15  local_bo_qty       242075 non-null  float64
 16  de

In [10]:
X_test = dataset.iloc[:,:-1]
y_test = dataset.went_on_backorder
scaler = MinMaxScaler().fit(X_test)
X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)

In [11]:
X_test.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,0.002089,0.153846,0.0,0.000013,0.000009,0.000006,0.00000,0.000000,0.00000,0.000000,...,1.0,0.0,0.9950,0.9952,0.0,0.0,1.0,1.0,1.0,1.0
1,0.002089,0.230769,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,1.0,0.0,0.9948,0.9948,0.0,0.0,1.0,1.0,0.0,1.0
2,0.002088,0.038462,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,1.0,0.0,0.9999,0.9996,0.0,1.0,1.0,1.0,0.0,1.0
3,0.002090,0.038462,0.0,0.000017,0.000032,0.000035,0.00002,0.000032,0.00003,0.000031,...,1.0,0.0,0.9996,0.9996,0.0,1.0,1.0,1.0,0.0,1.0
4,0.002088,0.153846,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,1.0,0.0,0.9978,0.9975,0.0,1.0,1.0,1.0,0.0,1.0


In [12]:
del dataset

We can now predict and evaluate with the preprocessed test set. It would be interesting to see the performance with and without outliers removal from the test set. We can report confusion matrix, precision, recall, f1-score, accuracy, and other measures (if any). 

In [36]:
predicted_y = model.predict(X_test)
print('Test set whole results:')
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test,predicted_y))

Test set whole results:
              precision    recall  f1-score   support

           0       0.02      0.91      0.04      2688
           1       1.00      0.50      0.67    239387

    accuracy                           0.51    242075
   macro avg       0.51      0.70      0.35    242075
weighted avg       0.99      0.51      0.66    242075



,0,1
0,2436,252
1,119175,120212


In [13]:
# Add code below this comment  (Question #E303)
# ----------------------------------

lof = LocalOutlierFactor(novelty=False).fit(X_test,y_test)
lof_outliers = lof.fit_predict(X_test)==-1
X_test_clean3 = X_test[~lof_outliers]
y_test_clean3 = y_test[~lof_outliers]














In [14]:
joblib.dump([X_test_clean3,y_test_clean3],'testsetcleaned-v1.pkl')

['testsetcleaned-v1.pkl']

In [37]:
predicted_y_clean = model.predict(X_test_clean3)
print('Outliers removed results:')
print(classification_report(y_test_clean3, predicted_y_clean))
pd.DataFrame(confusion_matrix(y_test_clean3,predicted_y_clean))

Outliers removed results:
              precision    recall  f1-score   support

           0       0.02      0.89      0.03      1464
           1       1.00      0.53      0.69    170123

    accuracy                           0.53    171587
   macro avg       0.51      0.71      0.36    171587
weighted avg       0.99      0.53      0.68    171587



,0,1
0,1306,158
1,80769,89354


In [23]:
import sys
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
temp = sorted([(x, sys.getsizeof(globals().get(x))/1024)
        for x in dir() 
        if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
df_temp = pd.DataFrame(temp, columns = ['variable', 'size_in_kilobytes'])
df_temp.head(20)

,variable,size_in_kilobytes
0,X_test,41606.664062
1,X_test_clean3,29491.539062
2,y_test,3782.445312
3,y_test_clean3,2681.070312
4,predicted_y,1891.304688
5,predicted_y_clean,1340.617188
6,lof_outliers,236.495117
7,LocalOutlierFactor,1.953125
8,DecisionTreeClassifier,1.429688
9,MinMaxScaler,1.156250


## Conclusion

## Reflect

Imagine you are data scientist that has been tasked with developing a system to save your 
company money by predicting and preventing back orders of parts in the supply chain.

Write a **brief summary** for "management" that details your findings, 
your level of certainty and trust in the models, 
and recommendations for operationalizing these models for the business.

# Save your notebook!
## Then `File > Close and Halt`